In [ ]:
!pip install arch

In [ ]:
!pip install --upgrade matplotlib statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 46.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.0
    Uninstalling matplotlib-3.8.0:
      Successfully uninstalled matplotlib-3.8.0


In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import yfinance as yf
import warnings
#from arch import arch_model
from statsmodels.tsa.stattools import adfuller

# Suppress warnings
warnings.filterwarnings("ignore")

# Load the Excel file
file_path = '/content/T-Bills_2024 (3).xlsx'

# Load stock data from Yahoo Finance
tickers = ['J&KBANK.NS', 'JAICORPLTD.NS', 'JAYNECOIND.NS', 'JCHAC.NS', '^NSEI']
df = pd.DataFrame()

start_date = '2020-04-01'
end_date = '2024-09-30'

# Download stock data
for ticker in tickers:
    tick = yf.Ticker(ticker)
    stock_data = tick.history(start=start_date, end=end_date)['Close']
    df[ticker] = stock_data
df2 = df.copy()

# Resample the stock data to daily, weekly, and monthly frequencies
df_daily = df.resample('D').ffill().pct_change().dropna().reset_index()
df_weekly = df.resample('W').ffill().pct_change().dropna().reset_index()
df_monthly = df.resample('MS').ffill().pct_change().dropna().reset_index()

# Convert comparison dates to timezone-aware datetime objects
start_date_daily = pd.to_datetime('2020-04-02').tz_localize('Asia/Kolkata')
start_date_weekly = pd.to_datetime('2020-04-04').tz_localize('Asia/Kolkata')
start_date_monthly = pd.to_datetime('2020-03-01').tz_localize('Asia/Kolkata')

# Adjust resampling start dates with timezone-aware comparison dates
df_daily = df_daily[df_daily['Date'] >= start_date_daily]
df_weekly = df_weekly[df_weekly['Date'] >= start_date_weekly]
df_monthly = df_monthly[df_monthly['Date'] >= start_date_monthly]

# Load the risk-free rate data
daily_rf = pd.read_excel(file_path, sheet_name='Daily')
weekly_rf = pd.read_excel(file_path, sheet_name='Weekly')
monthly_rf = pd.read_excel(file_path, sheet_name='Monthly')

# Convert 'Date' to datetime for the risk-free rate data
daily_rf['Date'] = pd.to_datetime(daily_rf['Date']).dt.tz_localize('Asia/Kolkata')
weekly_rf['Date'] = pd.to_datetime(weekly_rf['Date']).dt.tz_localize('Asia/Kolkata')
monthly_rf['Date'] = pd.to_datetime(monthly_rf['Date']).dt.tz_localize('Asia/Kolkata')

# Merge the risk-free rates (Daily, Weekly, Monthly) with stock data
df_daily = pd.merge(df_daily, daily_rf, on='Date', how='left')
df_weekly = pd.merge(df_weekly, weekly_rf, on='Date', how='left')
df_monthly = pd.merge(df_monthly, monthly_rf, on='Date', how='left')

# Add columns for daily, weekly, and monthly yield
df_daily['Daily Yield'] = df_daily['Daily Yield %']
df_weekly['Weekly Yield'] = df_weekly['Weekly Yield %']
df_monthly['Monthly Yield'] = df_monthly['Monthly Yield %']
